tgb - 4/13/2022 - The goal of this notebook is to train both brute-force and climate-invariant neural nets on (-4K) and (+4K) simultaneously to see whether climate-invariant nets facilitate learning both climates simultaneously. We will explore uncertainty using dropout and test different architectures by varying the depth of the NN.

# Imports and initialization

## Imports

In [1]:
from cbrain.climate_invariant import *
from cbrain.climate_invariant_utils import *
import pickle

from cbrain.imports import *
from cbrain.utils import *
from cbrain.normalization import *

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


## Initialization

In [2]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [3]:
path_train_m4K = path_data + '2021_03_18_O3_TRAIN_M4K_shuffle.nc'
path_train_p4K = path_data + '2021_03_18_O3_TRAIN_P4K_shuffle.nc'

In [4]:
path_valid_m4K = path_data + '2021_03_18_O3_VALID_M4K.nc'
path_valid_p4K = path_data + '2021_03_18_O3_VALID_P4K.nc'

In [5]:
path_test_m4K = path_data + '2021_03_18_O3_TEST_M4K.nc'
path_test_p4K = path_data + '2021_03_18_O3_TEST_P4K.nc'

In [6]:
path_input_norm = path_data + '2021_01_24_NORM_O3_small.nc'

In [7]:
path_norm_RH = path_data + '2021_02_01_NORM_O3_RH_small.nc'
path_norm_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'
path_norm_LHF_nsDELQ = path_data + '2021_02_01_NORM_O3_LHF_nsDELQ_small.nc'

In [8]:
in_vars = ['QBP','TBP','PS','SOLIN','SHFLX','LHFLX'] # We take the large-scale climate state as inputs
out_vars = ['PHQ','TPHYSTND','QRL','QRS'] # and we output the response of clouds/storms to these climate conditions

In [9]:
scale_dict = pickle.load(open(path_data+'009_Wm2_scaling.pkl','rb'))

# Data Generator

## Adapt the data generator so that it can load two datasets

In [10]:
class DataGenerator_2DS(tf.keras.utils.Sequence):
    """
    https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

    Data generator class.
    """

    def __init__(self, data_fn1, data_fn2, input_vars, output_vars,
                 norm_fn=None, input_transform=None, output_transform=None,
                 batch_size=8192, shuffle=True, xarray=False, var_cut_off=None):
        # Just copy over the attributes
        self.data_fn1, self.data_fn2, self.norm_fn = data_fn1, data_fn2, norm_fn
        self.input_vars, self.output_vars = input_vars, output_vars
        self.batch_size, self.shuffle = batch_size, shuffle

        # Open datasets
        self.data_ds = xr.open_mfdataset([self.data_fn1,self.data_fn2],
                                         concat_dim="sample",
                                         combine='nested')
        if norm_fn is not None: self.norm_ds = xr.open_dataset(norm_fn)

        # Compute number of samples and batches
        self.n_samples = self.data_ds.vars.shape[0]
        self.n_batches = int(np.floor(self.n_samples) / self.batch_size)

        # Get input and output variable indices
        self.input_idxs = return_var_idxs(self.data_ds, input_vars, var_cut_off)
        self.output_idxs = return_var_idxs(self.data_ds, output_vars)
        self.n_inputs, self.n_outputs = len(self.input_idxs), len(self.output_idxs)

        # Initialize input and output normalizers/transformers
        if input_transform is None:
            self.input_transform = Normalizer()
        elif type(input_transform) is tuple:
            self.input_transform = InputNormalizer(
                self.norm_ds, input_vars, input_transform[0], input_transform[1], var_cut_off)
        else:
            self.input_transform = input_transform  # Assume an initialized normalizer is passed

        if output_transform is None:
            self.output_transform = Normalizer()
        elif type(output_transform) is dict:
            self.output_transform = DictNormalizer(self.norm_ds, output_vars, output_transform)
        else:
            self.output_transform = output_transform  # Assume an initialized normalizer is passed

        # Now close the xarray file and load it as an h5 file instead
        # This significantly speeds up the reading of the data...
        if not xarray:
            self.data_ds.close()
            self.data_ds = h5py.File(data_fn1, 'r')

    def __len__(self):
        return self.n_batches

    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]

        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]

        # Normalize
        X = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)

        return X, Y

    def on_epoch_end(self):
        self.indices = np.arange(self.n_batches)
        if self.shuffle: np.random.shuffle(self.indices)

In [11]:
class DataGeneratorCI_2DS(DataGenerator_2DS):
    def __init__(self, data_fn1, data_fn2, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=8192, shuffle=True, xarray=False, var_cut_off=None, 
             Qscaling=None,
             Tscaling=None,
             LHFscaling=None,
             SHFscaling=None,
             output_scaling=False,
             interpolate=False,
             hyam=None,hybm=None,
             inp_sub_Qscaling=None,inp_div_Qscaling=None,
             inp_sub_Tscaling=None,inp_div_Tscaling=None,
             inp_sub_LHFscaling=None,inp_div_LHFscaling=None,
             inp_sub_SHFscaling=None,inp_div_SHFscaling=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,epsQ=1e-3,epsT=1,
                 mode='train'):
        self.output_scaling = output_scaling
        self.interpolate = interpolate
        self.Qscaling = Qscaling
        self.Tscaling = Tscaling
        self.LHFscaling = LHFscaling
        self.SHFscaling = SHFscaling
        self.inp_shape = 64
        self.mode=mode
        super().__init__(data_fn1, data_fn2, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        if Qscaling=='RH':
            self.QLayer = QV2RHNumpy(self.inp_sub,self.inp_div,inp_sub_Qscaling,inp_div_Qscaling,hyam,hybm)
        elif Qscaling=='QSATdeficit':
            self.QLayer = QV2QSATdeficitNumpy(self.inp_sub,self.inp_div,inp_sub_Qscaling,inp_div_Qscaling,hyam,hybm)
        if Tscaling=='TfromNS':
            self.TLayer = T2TmTNSNumpy(self.inp_sub,self.inp_div,inp_sub_Tscaling,inp_div_Tscaling,hyam,hybm)
        elif Tscaling=='BCONS':
            self.TLayer = T2BCONSNumpy(self.inp_sub,self.inp_div,inp_sub_Tscaling,inp_div_Tscaling,hyam,hybm)
        elif Tscaling=='BMSE':
            self.TLayer = T2BMSENumpy(self.inp_sub,self.inp_div,inp_sub_Tscaling,inp_div_Tscaling,hyam,hybm)
        elif Tscaling=='T_NSto220':
            self.TLayer = T2T_NSto220Numpy(self.inp_sub,self.inp_div,inp_sub_Tscaling,inp_div_Tscaling,hyam,hybm)
        if LHFscaling=='LHF_nsDELQ':
            self.LHFLayer = LHF2LHF_nsDELQNumpy(self.inp_sub,self.inp_div,inp_sub_LHFscaling,inp_div_LHFscaling,hyam,hybm,epsQ)
        elif LHFscaling=='LHF_nsQ':
            self.LHFLayer = LHF2LHF_nsQNumpy(self.inp_sub,self.inp_div,inp_sub_LHFscaling,inp_div_LHFscaling,hyam,hybm,epsQ)
        if SHFscaling=='SHF_nsDELT':
            self.SHFLayer = SHF2SHF_nsDELTNumpy(self.inp_sub,self.inp_div,inp_sub_SHFscaling,inp_div_SHFscaling,hyam,hybm,epsT)
        if output_scaling:
            self.scalingLayer = ScalingNumpy(hyam,hybm)
            self.inp_shape += 1
        if interpolate:
            self.interpLayer = InterpolationNumpy(lev,is_continous,Tnot,lower_lim,interm_size)
            self.inp_shape += interm_size*2 + 4 + 30 ## 4 same as 60-64 and 30 for lev_tilde.size
    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size
        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]
        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        X_result = np.copy(X_norm)
        if self.Qscaling:
            X_result = self.QLayer.process(X_result)
        if self.Tscaling:
            # tgb - 3/21/2021 - BCONS needs qv in kg/kg as an input
            if self.Tscaling=='BCONS' or self.Tscaling=='BMSE':
                if self.Qscaling:
                    X_resultT = self.TLayer.process(X_norm)
                    X_result = np.concatenate([X_result[:,:30],X_resultT[:,30:60],X_result[:,60:]], axis=1)
                else:
                    X_result = self.TLayer.process(X_result)
            else:
                X_result = self.TLayer.process(X_result)
        if self.SHFscaling:
            X_result = self.SHFLayer.process(X_result)
        if self.LHFscaling:
            # tgb - 3/22/2021 - LHF_ns(DEL)Q needs qv in kg/kg and T in K
            if self.Qscaling or self.Tscaling:
                X_resultLHF = self.LHFLayer.process(X_norm)
                X_result = np.concatenate([X_result[:,:60],X_resultLHF[:,60:]],axis=1)
            else:
                X_result = self.LHFLayer.process(X_result)
        if self.output_scaling:
            scalings = self.scalingLayer.process(X)
            X_result = np.hstack((X_result,scalings))
        if self.interpolate:
            interpolated = self.interpLayer.process(X,X_result)
            X_result = np.hstack((X_result,interpolated))
        if self.mode=='val':
            return xr.DataArray(X_result), xr.DataArray(Y)
        return X_result,Y
    ##transforms the input data into the required format, take the unnormalized dataset
    def transform(self,X):
        X_norm = self.input_transform.transform(X)
        X_result = X_norm
        if self.Qscaling:
            X_result = self.QLayer.process(X_result)
        if self.Tscaling:
            X_result = self.TLayer.process(X_result)
        if self.SHFscaling:
            X_result = self.SHFLayer.process(X_result)
        if self.LHFscaling:
            X_result = self.LHFLayer.process(X_result)
        if self.scaling:
            scalings = self.scalingLayer.process(X)
            X_result = np.hstack((X_result,scalings))
        if self.interpolate:
            interpolated = self.interpLayer.process(X,X_result)
            X_result = np.hstack((X_result,interpolated))
        return X_result

## Adapt the generator so that it can only use a reduced amount of training data

## Build data generator

### Brute-force

In [12]:
train_gen_BF = DataGeneratorCI_2DS(
    data_fn1 = path_train_m4K,
    data_fn2 = path_train_p4K,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict
)

### Climate-Invariant 

In [13]:
def train_gen_rescaling(input_rescaling):
    return DataGeneratorCI_2DS(
        data_fn1 = path_train_m4K,
        data_fn2 = path_train_p4K,
        input_vars = input_rescaling,
        output_vars = out_vars,
        norm_fn = path_input_norm,
        input_transform = ('mean', 'maxrs'),
        output_transform = scale_dict)

In [14]:
train_gen_RH = train_gen_rescaling(in_vars)
train_gen_BMSE = train_gen_rescaling(in_vars)
train_gen_LHF_nsDELQ = train_gen_rescaling(in_vars)

In [15]:
train_gen_CI = DataGeneratorCI_2DS(
    data_fn1 = path_train_m4K,
    data_fn2 = path_train_p4K,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean','maxrs'),
    output_transform = scale_dict,
    Qscaling = 'RH',
    Tscaling = 'BMSE',
    LHFscaling = 'LHF_nsDELQ',
    hyam=hyam, hybm=hybm, # Arrays to define mid-levels of hybrid vertical coordinate
    inp_sub_Qscaling=train_gen_RH.input_transform.sub, # What to subtract from RH inputs
    inp_div_Qscaling=train_gen_RH.input_transform.div, # What to divide RH inputs by
    inp_sub_Tscaling=train_gen_BMSE.input_transform.sub,
    inp_div_Tscaling=train_gen_BMSE.input_transform.div,
    inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
    inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div
)

### Validation set

In [16]:
valid_gen_BF = DataGeneratorCI_2DS(
    data_fn1 = path_valid_m4K,
    data_fn2 = path_valid_p4K,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict
)

In [17]:
valid_gen_CI = DataGeneratorCI_2DS(
    data_fn1 = path_valid_m4K,
    data_fn2 = path_valid_p4K,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean','maxrs'),
    output_transform = scale_dict,
    Qscaling = 'RH',
    Tscaling = 'BMSE',
    LHFscaling = 'LHF_nsDELQ',
    hyam=hyam, hybm=hybm, # Arrays to define mid-levels of hybrid vertical coordinate
    inp_sub_Qscaling=train_gen_RH.input_transform.sub, # What to subtract from RH inputs
    inp_div_Qscaling=train_gen_RH.input_transform.div, # What to divide RH inputs by
    inp_sub_Tscaling=train_gen_BMSE.input_transform.sub,
    inp_div_Tscaling=train_gen_BMSE.input_transform.div,
    inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
    inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div
)

### Test set

In [18]:
test_gen_BF = DataGeneratorCI_2DS(
    data_fn1 = path_test_m4K,
    data_fn2 = path_test_p4K,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict
)

In [19]:
test_gen_CI = DataGeneratorCI_2DS(
    data_fn1 = path_test_m4K,
    data_fn2 = path_test_p4K,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean','maxrs'),
    output_transform = scale_dict,
    Qscaling = 'RH',
    Tscaling = 'BMSE',
    LHFscaling = 'LHF_nsDELQ',
    hyam=hyam, hybm=hybm, # Arrays to define mid-levels of hybrid vertical coordinate
    inp_sub_Qscaling=train_gen_RH.input_transform.sub, # What to subtract from RH inputs
    inp_div_Qscaling=train_gen_RH.input_transform.div, # What to divide RH inputs by
    inp_sub_Tscaling=train_gen_BMSE.input_transform.sub,
    inp_div_Tscaling=train_gen_BMSE.input_transform.div,
    inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
    inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div
)

### Reduced data

# Models

In [20]:
def NN_model(inp,N_layer):
    if N_layer>0:
        densout = Dense(128, activation='linear')(inp)
        densout = LeakyReLU(alpha=0.3)(densout)
    else: dense_out = Dense(120, activation='linear')(inp)
    for i in range (N_layer-1):
        densout = Dense(128, activation='linear')(densout)
        densout = LeakyReLU(alpha=0.3)(densout)
    if N_layer>0: dense_out = Dense(120, activation='linear')(densout)
    return tf.keras.models.Model(inp, dense_out)

In [21]:
BF_models = {}
CI_models = {}
inp_BF = {}
inp_CI = {}

In [22]:
N_layer_max = 10

In [23]:
for ilayer in range(N_layer_max):
    
    print(ilayer)
    
    inp_BF[ilayer] = Input(shape=(64,))
    BF_models[ilayer] = NN_model(inp_BF[ilayer],ilayer)
    
    inp_CI[ilayer] = Input(shape=(64,))
    CI_models[ilayer] = NN_model(inp_CI[ilayer],ilayer)

0
1
2
3
4
5
6
7
8
9


In [24]:
CI_models[0].summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [25]:
for ilayer in range(N_layer_max):
    
    print(ilayer)
    
    BF_models[ilayer].compile(tf.keras.optimizers.Adam(), loss=mse)
    CI_models[ilayer].compile(tf.keras.optimizers.Adam(), loss=mse)

0
1
2
3
4
5
6
7
8
9


# Callbacks

In [26]:
class AdditionalValidationSets(Callback):
    def __init__(self, validation_sets, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            valid_generator,valid_name = validation_set
            results = self.model.evaluate_generator(generator=valid_generator)

            for metric, result in zip(self.model.metrics_names,[results]):
                valuename = valid_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

# Training

## Test on 2 layers only

In [124]:
ilayer = 2
Nep = 10

In [125]:
path_BF = path_data + '2022_04_13_BF_'+str(ilayer)+'.hdf5'

In [126]:
check_test = AdditionalValidationSets([(test_gen_BF,'Test MSE')])
mcp_save_BF = ModelCheckpoint(path_BF,save_best_only=True, monitor='val_loss', mode='min')
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

In [127]:
history_BF = BF_models[ilayer].fit_generator(train_gen_BF, epochs=Nep, 
                           validation_data = valid_gen_BF,
                           callbacks=[mcp_save_BF,check_test,earlyStopping])

Epoch 1/10
11518/11518 [==============================] - 1928s 167ms/step - loss: 116.8236 - val_loss: 109.5967
Epoch 2/10
11518/11518 [==============================] - 930s 81ms/step - loss: 106.1786 - val_loss: 104.2871
Epoch 3/10
11518/11518 [==============================] - 915s 79ms/step - loss: 102.8043 - val_loss: 101.8122
Epoch 4/10
11518/11518 [==============================] - 916s 79ms/step - loss: 100.9116 - val_loss: 100.2472
Epoch 5/10
11518/11518 [==============================] - 916s 79ms/step - loss: 99.4301 - val_loss: 100.5478
Epoch 6/10
11518/11518 [==============================] - 917s 80ms/step - loss: 98.3662 - val_loss: 98.1168
Epoch 7/10
11518/11518 [==============================] - 885s 77ms/step - loss: 97.6653 - val_loss: 98.8207
Epoch 8/10
11518/11518 [==============================] - 900s 78ms/step - loss: 96.9073 - val_loss: 96.6271
Epoch 9/10
11518/11518 [==============================] - 940s 82ms/step - loss: 96.3519 - val_loss: 99.0366
Epoch 10

In [128]:
path_CI = path_data + '2022_04_13_CI_'+str(ilayer)+'.hdf5'

In [131]:
check_test = AdditionalValidationSets([(test_gen_CI,'Test MSE')])
mcp_save = ModelCheckpoint(path_CI,save_best_only=True, monitor='val_loss', mode='min')
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

In [ ]:
history_CI = CI_models[ilayer].fit_generator(train_gen_CI, epochs=Nep, 
                           validation_data = valid_gen_CI,
                           callbacks=[mcp_save,check_test,earlyStopping])

Epoch 1/10
11518/11518 [==============================] - 2584s 224ms/step - loss: 120.2141 - val_loss: 109.7997
Epoch 2/10
11518/11518 [==============================] - 2618s 227ms/step - loss: 109.6423 - val_loss: 109.7831
Epoch 3/10
11518/11518 [==============================] - 2668s 232ms/step - loss: 106.6781 - val_loss: 103.8799
Epoch 4/10
11518/11518 [==============================] - 2517s 219ms/step - loss: 104.6863 - val_loss: 102.3276
Epoch 5/10
 3201/11518 [=======>......................] - ETA: 12:00 - loss: 104.8467

## Train one BF and one CI model per number of layers

In [31]:
updated_range = np.arange(3,N_layer_max)

In [32]:
updated_range

array([3, 4, 5, 6, 7, 8, 9])

In [33]:
Nep = 20

In [34]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
check_test_BF = AdditionalValidationSets([(test_gen_BF,'Test MSE')])
check_test_CI = AdditionalValidationSets([(test_gen_CI,'Test MSE')])

In [35]:
history_BF = {}
history_CI = {}

In [36]:
for ilayer in updated_range:
    
    print('Number of layers = ',ilayer)
    
    path_BF = path_data + '2022_04_14_BF_'+str(ilayer)+'.hdf5'
    mcp_save = ModelCheckpoint(path_BF,save_best_only=True, monitor='val_loss', mode='min')
    history_BF[ilayer] = BF_models[ilayer].fit_generator(train_gen_BF, epochs=Nep, 
                                                         validation_data = valid_gen_BF,
                                                         callbacks=[mcp_save,earlyStopping])
    
    path_CI = path_data + '2022_04_14_CI_'+str(ilayer)+'.hdf5'
    mcp_save = ModelCheckpoint(path_CI,save_best_only=True, monitor='val_loss', mode='min')
    history_CI[ilayer] = CI_models[ilayer].fit_generator(train_gen_CI, epochs=Nep, 
                                                         validation_data = valid_gen_CI,
                                                         callbacks=[mcp_save,earlyStopping])

Number of layers =  3
Epoch 1/20
11518/11518 [==============================] - 719s 62ms/step - loss: 112.9407 - val_loss: 113.3926
Epoch 2/20
11518/11518 [==============================] - 701s 61ms/step - loss: 104.0487 - val_loss: 101.9438
Epoch 3/20
11518/11518 [==============================] - 690s 60ms/step - loss: 100.2444 - val_loss: 100.0977
Epoch 4/20
11518/11518 [==============================] - 695s 60ms/step - loss: 97.8477 - val_loss: 96.6934
Epoch 5/20
11518/11518 [==============================] - 680s 59ms/step - loss: 96.5323 - val_loss: 95.6660
Epoch 6/20
11518/11518 [==============================] - 688s 60ms/step - loss: 95.3004 - val_loss: 94.1513
Epoch 7/20
11518/11518 [==============================] - 693s 60ms/step - loss: 94.4471 - val_loss: 93.7933
Epoch 8/20
11518/11518 [==============================] - 694s 60ms/step - loss: 93.6100 - val_loss: 92.5446
Epoch 9/20
11518/11518 [==============================] - 686s 60ms/step - loss: 92.9777 - val_loss:

11518/11518 [==============================] - 1816s 158ms/step - loss: 102.2888 - val_loss: 100.0648
Epoch 8/20
11518/11518 [==============================] - 2009s 174ms/step - loss: 102.5505 - val_loss: 98.9804
Epoch 9/20
11518/11518 [==============================] - 1905s 165ms/step - loss: 100.4476 - val_loss: 99.2355
Epoch 10/20
11518/11518 [==============================] - 2074s 180ms/step - loss: 99.8990 - val_loss: 106.7556
Epoch 11/20
11518/11518 [==============================] - 1697s 147ms/step - loss: 99.6918 - val_loss: 97.7775
Epoch 12/20
11518/11518 [==============================] - 1854s 161ms/step - loss: 99.1390 - val_loss: 97.6088
Epoch 13/20
11518/11518 [==============================] - 1810s 157ms/step - loss: 99.3583 - val_loss: 97.1940
Epoch 14/20
11518/11518 [==============================] - 1829s 159ms/step - loss: 99.1550 - val_loss: 96.8473
Epoch 15/20
11517/11518 [============================>.] - ETA: 0s - loss: 98.7425

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



## Using the best architecture for BF/CI neural nets, logarithmically increase data size

# Evaluation: Final evaluation on training/validation/test set and make plot

Evaluate for each # of layers, for each training/validation/test set on joint/-4K/+4K datasets. This should amount to 90 evaluations. 

## Different number of layers

## Different training data size